## Feature Engineering
- time of each trip hr
- length of each trip in km
- 

In [33]:
from haversine import Unit
import haversine as hs
import pandas as pd


In [34]:
import sys
sys.path.append('../scripts/')

from data_cleaning import DataCleaning
from data_extraction import DataExtraction
from feature_extraction import FeatureExtraction
dclean = DataCleaning()
dextract = DataExtraction()
fextract = FeatureExtraction()


In [35]:
clean_df = dextract.load_data('../data/clean_data.csv')
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,order_origin,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [36]:
clean_df = fextract.cordinate_tupple(clean_df,['order_origin','trip_origin','trip_destination'])
clean_df = fextract.convert_to_date(clean_df,['trip_Start_time','trip_end_time'])

In [37]:
features = clean_df.copy()

- calculate distance from the order origin to the driver origin
- calculate trip distance
- calculate trip duration

In [38]:

features['driver_order_dst'] = [fextract.distance_from(loc1,loc2) for loc1, loc2 in zip(clean_df.order_origin,clean_df.trip_origin)]
features['trip_dstance'] = [fextract.distance_from(loc1,loc2) for loc1, loc2 in zip(clean_df.trip_origin,clean_df.trip_destination)] 
features['trip_time'] = (clean_df.trip_end_time - clean_df.trip_Start_time).astype('timedelta64[s]')
features[['driver_order_dst', 'trip_dstance','trip_time']].head()

,driver_order_dst,trip_dstance,trip_time
0,693.62,21048.29,217.0
1,1554.09,21048.29,217.0
2,788.35,21048.29,217.0
3,694.07,21048.29,217.0
4,1623.33,21048.29,217.0


- calculate year/month/weekdays

In [39]:
# features['year'] = clean_df['trip_Start_time'].dt.year
features = fextract.get_ymwdh(features,'trip_Start_time')
features[['month','day','week_day','hour']]

,month,day,week_day,hour
0,7.0,1.0,3.0,9.0
1,7.0,1.0,3.0,9.0
2,7.0,1.0,3.0,9.0
3,7.0,1.0,3.0,9.0
4,7.0,1.0,3.0,9.0
...,...,...,...,...
1557735,9.0,10.0,4.0,11.0
1557736,9.0,10.0,4.0,11.0
1557737,9.0,10.0,4.0,11.0
1557738,9.0,10.0,4.0,11.0


## get weahter

In [40]:
# importing the requests library
import requests
  
# api-endpoint
URL = "https://api.weatherbit.io/v2.0/history/daily?"


In [44]:
weather = {'clouds':[],
           'dewpt':[],
           'wind_spd':[],
           'temp':[],
           'precip':[],
           'snow':[]}

In [47]:
for i in range(78,features.shape[0]):
    # defining a params dict for the parameters to be sent to the API
    PARAMS = {
        'lat':features.lat.iloc[i], 
        'lon':features.lng.iloc[i],
        'start_date':str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour),
        'end_date':str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour+24),
        'key':'833701327e23466183824f3bbb10bcad'
        }
    # print(str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour),) 
    # sending get request and saving the response as response object
    r = requests.get(url = URL, params = PARAMS)
    
    # extracting data in json format
    data = r.json()
  
    # extracting latitude, longitude and formatted address 
    # of the first matching location
    weather['id'] = features['id'].iloc[i]
    weather['clouds'].append(data['data'][0]['clouds'])
    weather['dewpt'].append(data['data'][0]['dewpt'])
    weather['wind_spd'].append(data['data'][0]['wind_spd'])
    weather['temp'].append(data['data'][0]['temp'])
    weather['precip'].append(data['data'][0]['precip'])
    weather['snow'].append(data['data'][0]['snow'])
    

In [46]:
weather_df = pd.DataFrame(weather)
print(weather_df.shape)
weather_df.head()

(77, 7)


,clouds,dewpt,wind_spd,temp,precip,snow,id
0,81,24.1,2.9,27.8,2.8,0,77
1,81,24.1,2.9,27.8,2.8,0,77
2,81,24.1,2.9,27.8,2.8,0,77
3,81,24.1,2.9,27.8,2.8,0,77
4,81,24.1,2.9,27.8,2.8,0,77


In [ ]:
features.to_csv('../data/features.csv',index=False)